In [155]:
import numpy as np
import pandas as pd
import os

In [156]:
df1 = pd.read_csv('../Data/Clean/USAFacts/COVIDcounts_county_clean.csv', index_col = 0)
print(df1.shape)
df1.head()

(3142, 895)


,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,5813,5818,5825,5832,5832,5832,5857,5863,5880,5891
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,6630,6648,6654,6663,6663,6663,6679,6687,6700,6704
5,1009,Blount County,AL,1,0,0,0,0,0,0,...,15246,15249,15272,15283,15283,15283,15324,15335,15351,15370


In [157]:
for index, column in enumerate(df1.columns[:]):
    if column.startswith('2020-12-30'):
        labs20 = column
    if column.startswith('2021-12-30'):
        labs21 = column
    if column.startswith('2022-06-30'):
        labs22 = column

print(labs20)
print(labs21)
print(labs22)

2020-12-30
2021-12-30
2022-06-30


In [158]:
df1 = pd.DataFrame(np.repeat(df1.values, 3, axis = 0), columns = df1.columns)
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905


In [159]:
df1['year'] = None
for index in df1.index:
    if index % 3 == 0:
        df1.iloc[index, -1] = 2020
    elif index % 3 == 1:
        df1.iloc[index, -1] = 2021
    else:
        df1.iloc[index, -1] = 2022
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2020
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2021
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2022
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58169,58293,58372,58372,58372,58595,58695,58796,58905,2020
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58169,58293,58372,58372,58372,58595,58695,58796,58905,2021


In [160]:
df1['covid_cases'] = None
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year,covid_cases
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2020,None
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2021,None
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2022,None
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2020,None
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2021,None


In [161]:
for index in df1.index:
    if index % 3 == 0:
        df1.iloc[index,-1] = df1.loc[index,labs20]
    elif index % 3 == 1:
        df1.iloc[index,-1] = df1.loc[index,labs21]
    else:
        df1.iloc[index,-1] = df1.loc[index,labs22]
df1

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year,covid_cases
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2020,4164
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2021,10953
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2022,16536
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2020,13392
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2021,39583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9421,56043,Washakie County,WY,56,0,0,0,0,0,0,...,2490,2490,2490,2490,2490,2482,2482,2482,2021,1880
9422,56043,Washakie County,WY,56,0,0,0,0,0,0,...,2490,2490,2490,2490,2490,2482,2482,2482,2022,2482
9423,56045,Weston County,WY,56,0,0,0,0,0,0,...,1641,1641,1641,1641,1641,1643,1643,1643,2020,475
9424,56045,Weston County,WY,56,0,0,0,0,0,0,...,1641,1641,1641,1641,1641,1643,1643,1643,2021,1254


In [162]:
df = df1[['year', 'fips','covid_cases', 'county_name']]
print(df.dtypes)
df[['year','fips','covid_cases']] = df[['year','fips','covid_cases']].apply(pd.to_numeric)
print(df.dtypes)
df

year           object
fips           object
covid_cases    object
county_name    object
dtype: object
year            int64
fips            int64
covid_cases     int64
county_name    object
dtype: object


C:\Users\FPSKINSMANC\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,year,fips,covid_cases,county_name
0,2020,1001,4164,Autauga County
1,2021,1001,10953,Autauga County
2,2022,1001,16536,Autauga County
3,2020,1003,13392,Baldwin County
4,2021,1003,39583,Baldwin County
...,...,...,...,...
9421,2021,56043,1880,Washakie County
9422,2022,56043,2482,Washakie County
9423,2020,56045,475,Weston County
9424,2021,56045,1254,Weston County


In [163]:
df.to_csv('../Data/Intermediate/COVIDcounty_cases_processed.csv', index = False)

In [164]:
dp03, dp05 = [], []
path = '../Data/Clean/ACS'
for file in os.listdir(path):
    if file.find('DP03') != -1:
        print(file)
        dp03.append(pd.read_csv(os.path.join(path, file), index_col = 0))
    if file.find('DP05') != -1:
        print(file)
        dp05.append(pd.read_csv(os.path.join(path, file), index_col = 0))

i = 0
while i != len(dp03):
    dp03[i].drop(columns = ['geographic_area_name','id'], inplace = True)
    print('03: ', dp03[i].shape)
    dp05[i].drop(columns = ['Geographic Area Name','id'], inplace = True)
    print('05: ',dp05[i].shape)
    i += 1

ACSDP032010_clean.csv
ACSDP032011_clean.csv
ACSDP032012_clean.csv
ACSDP032013_clean.csv
ACSDP032014_clean.csv
ACSDP032015_clean.csv
ACSDP032016_clean.csv
ACSDP032017_clean.csv
ACSDP032018_clean.csv
ACSDP032019_clean.csv
ACSDP032020_clean.csv
ACSDP052010_clean.csv
ACSDP052011_clean.csv
ACSDP052012_clean.csv
ACSDP052013_clean.csv
ACSDP052014_clean.csv
ACSDP052015_clean.csv
ACSDP052016_clean.csv
ACSDP052017_clean.csv
ACSDP052018_clean.csv
ACSDP052019_clean.csv
ACSDP052020_clean.csv
03:  (3221, 20)
05:  (3221, 19)
03:  (3221, 20)
05:  (3221, 19)
03:  (3221, 20)
05:  (3221, 19)
03:  (3221, 20)
05:  (3221, 33)
03:  (3220, 20)
05:  (3220, 33)
03:  (3220, 20)
05:  (3220, 33)
03:  (3220, 20)
05:  (3220, 33)
03:  (3220, 20)
05:  (3220, 34)
03:  (3220, 20)
05:  (3220, 34)
03:  (3220, 20)
05:  (3220, 34)
03:  (3221, 20)
05:  (3221, 34)


In [165]:
i = 2
tot_dp03 = pd.concat([dp03[0], dp03[1]], ignore_index = True)
tot_dp05 = pd.concat([dp05[0], dp05[1]], ignore_index = True)

while i != len(dp03):
    tot_dp03 = pd.concat([tot_dp03, dp03[i]], ignore_index = True)
    tot_dp05 = pd.concat([tot_dp05, dp05[i]], ignore_index = True)
    i += 1

print(tot_dp03.shape)
print(tot_dp05.shape)
tot_dp03

(35425, 31)
(35425, 42)


,per_capita_income_2010,unemployment_rate,employment_pop_ratio,pct_ind_agriculture_forestry_fishing_and_hunting_and_mining,pct_ind_construction,pct_ind_manufacturing,pct_ind_wholesale_trade,pct_ind_retail_trade,pct_ind_transportation_and_warehousing_and_utilities,pct_ind_information,...,per_capita_income_2012,per_capita_income_2013,pct_employment_status!!population_16_years_and_over!!in_labor_force!!civilian_labor_force!!unemployed,per_capita_income_2014,per_capita_income_2015,per_capita_income_2016,per_capita_income_2017,per_capita_income_2018,per_capita_income_2019,per_capita_income_2020
0,24568.0,4.0,60.4,0.9,7.7,14.1,3.1,12.7,4.4,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,26469.0,3.9,56.2,1.9,10.7,9.2,3.7,13.8,5.4,1.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15875.0,4.7,44.6,4.1,6.3,28.3,1.3,12.5,7.9,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19918.0,5.5,54.7,3.8,10.6,18.0,2.3,13.7,10.4,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21070.0,4.5,55.7,3.2,11.0,16.8,3.8,11.6,7.0,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35420,NaN,18.1,35.1,1.0,6.2,12.5,4.2,11.0,3.9,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11444.0
35421,NaN,13.1,29.0,0.7,18.4,2.6,0.0,1.0,5.4,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9096.0
35422,NaN,20.0,36.1,3.9,7.1,17.7,0.4,10.1,2.9,0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11596.0
35423,NaN,18.1,31.2,0.6,5.1,19.6,1.0,11.7,3.4,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10124.0


In [166]:
tot_dp05.columns

Index(['pct_sex_age_male', 'pct_sex_age_female', 'pct_sex_age_85+',
       'median_age', 'pct_sex_age_18+', 'pct_sex_age_21+', 'pct_sex_age_62+',
       'pct_sex_age_65+', 'pct_one_or_more_race', 'pct_white', 'pct_black',
       'pct_native_american', 'pct_asian',
       'pct_native_hawaiian_pacific_islander', 'pct_other_race',
       'pct_ethnicity_hispanic', 'year', 'total_pop', 'fips',
       'pct_sex_age_under_5', 'pct_sex_age_5_to_9', 'pct_sex_age_10_to_14',
       'pct_sex_age_15_to_19', 'pct_sex_age_20_to_24', 'pct_sex_age_25_to_34',
       'pct_sex_age_35_to_44', 'pct_sex_age_45_to_54', 'pct_sex_age_55_to_59',
       'pct_sex_age_60_to_64', 'pct_sex_age_65_to_74', 'pct_sex_age_75_to_84',
       'pct_sex_age_18+_male', 'pct_sex_age_18+_female',
       'pct_sex_age_65+_male', 'pct_sex_age_65+_female',
       'pct_one_or_more_race_white', 'pct_one_or_more_race_black',
       'pct_one_or_more_race_native_american', 'pct_one_or_more_race_asian',
       'pct_one_or_more_race_native_h

In [167]:
tot_dp05

,pct_sex_age_male,pct_sex_age_female,pct_sex_age_85+,median_age,pct_sex_age_18+,pct_sex_age_21+,pct_sex_age_62+,pct_sex_age_65+,pct_one_or_more_race,pct_white,...,pct_sex_age_18+_female,pct_sex_age_65+_male,pct_sex_age_65+_female,pct_one_or_more_race_white,pct_one_or_more_race_black,pct_one_or_more_race_native_american,pct_one_or_more_race_asian,pct_one_or_more_race_native_hawaiian_pacific_islander,pct_one_or_more_race_other_race,pct_sex_age_16+
0,48.5,51.5,1.1,36.2,72.3,67.9,14.3,11.4,53155,80.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48.9,51.1,1.6,41.0,76.7,73.3,20.1,16.5,175791,87.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52.9,47.1,1.8,38.0,77.5,73.7,16.7,13.7,27699,50.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53.8,46.2,0.7,38.3,76.6,72.5,15.7,12.4,22610,84.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49.5,50.5,1.2,38.3,75.2,71.3,18.1,14.2,56692,96.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35420,NaN,NaN,2.6,42.4,81.0,77.0,24.6,20.3,51089,NaN,...,53.0,44.5,55.5,82.2,5.8,0.1,0.0,0.0,20.7,83.7
35421,NaN,NaN,4.7,43.8,80.9,76.1,26.2,22.1,8508,NaN,...,49.7,47.4,52.6,45.6,12.5,0.1,0.2,0.0,44.1,83.3
35422,NaN,NaN,1.1,41.3,79.2,74.7,21.3,17.7,21899,NaN,...,52.5,45.2,54.8,85.0,29.8,0.2,0.0,0.0,15.8,82.3
35423,NaN,NaN,2.6,44.1,81.5,77.5,25.4,20.9,32867,NaN,...,52.4,46.0,54.0,15.4,68.8,0.1,0.0,0.0,17.8,84.2


In [168]:
tot_dp03.to_csv('../Data/Intermediate/ACSDP03_merge.csv')
tot_dp05.to_csv('../Data/Intermediate/ACSDP05_merge.csv')

In [169]:
tot_dp03.shape

(35425, 31)

In [170]:
hps = pd.read_csv('../Data/Clean/HPS/hps_clean.csv', index_col = 0)
print(hps.columns)

Index(['year', 'area', 'pct_delayed_healthcare'], dtype='object')


In [171]:
hps['state_fips'] = None
state = [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 
         21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 
         38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56]
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming']

dic = {state[i]: us_states[i] for i in range(len(state))}

print(len(state))
print(len(us_states))
print(dic)

hps['state_fips'] = dic
hps

51
51
{1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California', 8: 'Colorado', 9: 'Connecticut', 10: 'Delaware', 11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii', 16: 'Idaho', 17: 'Illinois', 18: 'Indiana', 19: 'Iowa', 20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland', 25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota', 28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska', 32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico', 36: 'New York', 37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio', 40: 'Oklahoma', 41: 'Oregon', 42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota', 47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington', 54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming'}


,year,area,pct_delayed_healthcare,state_fips
0,2020,Alabama,39.200000,1
1,2020,Alaska,45.708333,2
2,2020,Arizona,41.016667,4
3,2020,Arkansas,40.575000,5
4,2020,California,41.116667,6
5,2020,Colorado,41.158333,8
6,2020,Connecticut,41.391667,9
7,2020,Delaware,38.766667,10
8,2020,District of Columbia,43.500000,11
9,2020,Florida,40.216667,12


In [172]:
hps.to_csv('../Data/Intermediate/hps_processed.csv')